In [1]:
from random import seed
from random import randint
from numpy import array
from math import ceil
from math import log10
from math import sqrt
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector


In [2]:
from random import seed
from random import randint
import operator
import functools

seed(1)
X, y = list(), list()
for i in range(100):
    in_pattern = [randint(1,100) for _ in range(2)]
    out_pattern = sum(in_pattern)
    print(in_pattern, out_pattern)
    X.append(in_pattern)
    y.append(out_pattern)

[18, 73] 91
[98, 9] 107
[33, 16] 49
[64, 98] 162
[58, 61] 119
[84, 49] 133
[27, 13] 40
[63, 4] 67
[50, 56] 106
[78, 98] 176
[99, 1] 100
[90, 58] 148
[35, 93] 128
[30, 76] 106
[14, 41] 55
[4, 3] 7
[4, 84] 88
[70, 2] 72
[49, 88] 137
[28, 55] 83
[93, 4] 97
[68, 29] 97
[98, 57] 155
[64, 71] 135
[30, 45] 75
[30, 87] 117
[29, 98] 127
[59, 38] 97
[3, 54] 57
[72, 83] 155
[13, 24] 37
[81, 93] 174
[38, 16] 54
[96, 43] 139
[93, 92] 185
[65, 55] 120
[65, 86] 151
[25, 39] 64
[37, 76] 113
[64, 65] 129
[51, 76] 127
[5, 62] 67
[32, 96] 128
[52, 54] 106
[86, 23] 109
[47, 71] 118
[90, 100] 190
[87, 95] 182
[48, 12] 60
[57, 85] 142
[66, 14] 80
[100, 21] 121
[67, 51] 118
[48, 63] 111
[94, 4] 98
[61, 6] 67
[40, 91] 131
[79, 76] 155
[75, 51] 126
[83, 22] 105
[22, 65] 87
[30, 2] 32
[99, 26] 125
[70, 71] 141
[30, 52] 82
[66, 45] 111
[74, 46] 120
[59, 35] 94
[85, 71] 156
[78, 94] 172
[1, 50] 51
[95, 66] 161
[17, 67] 84
[100, 72] 172
[27, 55] 82
[8, 62] 70
[47, 73] 120
[71, 26] 97
[65, 53] 118
[63, 46] 109
[54,

In [3]:
from random import randint
from math import ceil
from math import log10
 
# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
	X, y = list(), list()
	for i in range(n_examples):
		in_pattern = [randint(1,largest) for _ in range(n_numbers)]
		out_pattern = sum(in_pattern)
		X.append(in_pattern)
		y.append(out_pattern)
	return X, y

In [4]:
# convert data to strings
def to_string(X, y, n_numbers, largest):
	max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
	Xstr = list()
	for pattern in X:
		strp = '+'.join([str(n) for n in pattern])
		strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
		Xstr.append(strp)
	max_length = ceil(log10(n_numbers * (largest+1)))
	ystr = list()
	for pattern in y:
		strp = str(pattern)
		strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
		ystr.append(strp)
	return Xstr, ystr

In [5]:
# integer encode strings
def integer_encode(X, y, alphabet):
	char_to_int = dict((c, i) for i, c in enumerate(alphabet))
	Xenc = list()
	for pattern in X:
		integer_encoded = [char_to_int[char] for char in pattern]
		Xenc.append(integer_encoded)
	yenc = list()
	for pattern in y:
		integer_encoded = [char_to_int[char] for char in pattern]
		yenc.append(integer_encoded)
	return Xenc, yenc

In [17]:
seed(1)
n_samples = 20
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)

[[3, 10], [2, 5], [2, 8], [8, 8], [7, 4], [2, 8], [1, 7], [7, 10], [1, 8], [5, 4], [10, 2], [6, 1], [1, 1], [9, 1], [7, 4], [7, 1], [9, 4], [8, 8], [9, 4], [6, 4]] [13, 7, 10, 16, 11, 10, 8, 17, 9, 9, 12, 7, 2, 10, 11, 8, 13, 16, 13, 10]
[' 3+10', '  2+5', '  2+8', '  8+8', '  7+4', '  2+8', '  1+7', ' 7+10', '  1+8', '  5+4', ' 10+2', '  6+1', '  1+1', '  9+1', '  7+4', '  7+1', '  9+4', '  8+8', '  9+4', '  6+4'] ['13', ' 7', '10', '16', '11', '10', ' 8', '17', ' 9', ' 9', '12', ' 7', ' 2', '10', '11', ' 8', '13', '16', '13', '10']
[[11, 3, 10, 1, 0], [11, 11, 2, 10, 5], [11, 11, 2, 10, 8], [11, 11, 8, 10, 8], [11, 11, 7, 10, 4], [11, 11, 2, 10, 8], [11, 11, 1, 10, 7], [11, 7, 10, 1, 0], [11, 11, 1, 10, 8], [11, 11, 5, 10, 4], [11, 1, 0, 10, 2], [11, 11, 6, 10, 1], [11, 11, 1, 10, 1], [11, 11, 9, 10, 1], [11, 11, 7, 10, 4], [11, 11, 7, 10, 1], [11, 11, 9, 10, 4], [11, 11, 8, 10, 8], [11, 11, 9, 10, 4], [11, 11, 6, 10, 4]] [[1, 3], [11, 7], [1, 0], [1, 6], [1, 1], [1, 0], [11, 8], [1,

In [18]:
# one hot encode
def one_hot_encode(X, y, max_int):
	Xenc = list()
	for seq in X:
		pattern = list()
		for index in seq:
			vector = [0 for _ in range(max_int)]
			vector[index] = 1
			pattern.append(vector)
		Xenc.append(pattern)
	yenc = list()
	for seq in y:
		pattern = list()
		for index in seq:
			vector = [0 for _ in range(max_int)]
			vector[index] = 1
			pattern.append(vector)
		yenc.append(pattern)
	return Xenc, yenc

In [19]:
# one hot encode
X, y = one_hot_encode(X, y, len(alphabet))
print(X, y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 

In [20]:
def generate_data(n_samples, n_numbers, largest, alphabet):
	# generate pairs
	X, y = random_sum_pairs(n_samples, n_numbers, largest)
	# convert to strings
	X, y = to_string(X, y, n_numbers, largest)
	# integer encode
	X, y = integer_encode(X, y, alphabet)
	# one hot encode
	X, y = one_hot_encode(X, y, len(alphabet))
	# return as numpy arrays
	X, y = array(X), array(y)
	return X, y

In [21]:
X

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [22]:
y

[[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0,

In [23]:
def invert(seq, alphabet):
	int_to_char = dict((i, c) for i, c in enumerate(alphabet))
	strings = list()
	for pattern in seq:
		string = int_to_char[argmax(pattern)]
		strings.append(string)
	return ''.join(strings)

In [25]:
from random import randint
from numpy import array
from math import ceil
from math import log10
from math import sqrt
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.layers import Masking
#n_chars = len(alphabet)
#n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
#n_out_seq_length = ceil(log10(n_numbers * (largest+1)))
# define LSTM configuration
n_batch = 10
n_epoch = 30
# create LSTM
model = Sequential()
model.add(LSTM(100, input_shape=(5,12)))

model.add(RepeatVector(2))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(12, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# train LSTM
for i in range(n_epoch):
	X, y = generate_data(n_samples, n_numbers, largest, alphabet)
	print(i)
	model.fit(X, y, epochs=n_epoch, batch_size=n_batch)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100)               45200     
                                                                 
 repeat_vector_4 (RepeatVect  (None, 2, 100)           0         
 or)                                                             
                                                                 
 lstm_9 (LSTM)               (None, 2, 50)             30200     
                                                                 
 time_distributed_4 (TimeDis  (None, 2, 12)            612       
 tributed)                                                       
                                                                 
Total params: 76,012
Trainable params: 76,012
Non-trainable params: 0
_________________________________________________________________
None
0
Epoch 1/30
2/2 [===========================

In [26]:
# evaluate on some new patterns
X, y = generate_data(n_samples, n_numbers, largest, alphabet)
result = model.predict(X, batch_size=n_batch, verbose=0)
# calculate error
expected = [invert(x, alphabet) for x in y]
predicted = [invert(x, alphabet) for x in result]
# show some examples
for i in range(20):
	print('Expected=%s, Predicted=%s' % (expected[i], predicted[i]))

Expected= 6, Predicted= 8
Expected=10, Predicted=10
Expected=13, Predicted=13
Expected= 6, Predicted= 5
Expected=14, Predicted=13
Expected=19, Predicted=19
Expected=12, Predicted=12
Expected= 7, Predicted= 5
Expected= 6, Predicted= 5
Expected=14, Predicted=13
Expected= 6, Predicted= 5
Expected=11, Predicted=10
Expected=15, Predicted=13
Expected=16, Predicted=15
Expected= 3, Predicted= 5
Expected=12, Predicted=12
Expected= 5, Predicted= 5
Expected= 8, Predicted= 8
Expected=11, Predicted=11
Expected= 8, Predicted= 8


In [27]:
X, y = list(), list()
in_pattern =input('dwse dyo arithmous me dyo kena anamesa tous')
print(in_pattern)
in_pattern=in_pattern.split()
print(in_pattern)
in_pattern = [ int(x) for x in in_pattern]
  
print(in_pattern)


dwse dyo arithmous me dyo kena anamesa tous5  7
5  7
['5', '7']
[5, 7]


In [28]:
out_pattern=sum(in_pattern)
print(in_pattern,out_pattern)
X.append(in_pattern)
y.append(out_pattern)

[5, 7] 12


In [29]:
from random import randint
from math import ceil
from math import log10
 
# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
	X, y = list(), list()
	for i in range(n_examples):
		in_pattern = [randint(1,largest) for _ in range(n_numbers)]
		out_pattern=sum(in_pattern)
		X.append(in_pattern)
		y.append(out_pattern)
	return X, y

In [30]:
# convert data to strings
def to_string(X, y, n_numbers, largest):
	max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
	Xstr = list()
	for pattern in X:
		strp = '+'.join([str(n) for n in pattern])
		strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
		Xstr.append(strp)
	max_length = ceil(log10(n_numbers * (largest+1)))
	ystr = list()
	for pattern in y:
		strp = str(pattern)
		strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
		ystr.append(strp)
	return Xstr, ystr

In [31]:
from random import seed
seed(1)
n_samples = 20
n_numbers = 2
largest = 10

# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)

['  5+7'] ['12']
[[11, 11, 5, 10, 7]] [[1, 2]]


In [32]:
# one hot encode
def one_hot_encode(X, y, max_int):
	Xenc = list()
	for seq in X:
		pattern = list()
		for index in seq:
			vector = [0 for _ in range(max_int)]
			vector[index] = 1
			pattern.append(vector)
		Xenc.append(pattern)
	yenc = list()
	for seq in y:
		pattern = list()
		for index in seq:
			vector = [0 for _ in range(max_int)]
			vector[index] = 1
			pattern.append(vector)
		yenc.append(pattern)
	return Xenc, yenc

In [33]:
# one hot encode
X, y = one_hot_encode(X, y, len(alphabet))
print(X, y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [34]:
def generate_data(n_samples, n_numbers, largest, alphabet):
	
	# convert to strings
	X, y = to_string(X, y, n_numbers, largest)
	# integer encode
	X, y = integer_encode(X, y, alphabet)
	# one hot encode
	X, y = one_hot_encode(X, y, len(alphabet))
	# return as numpy arrays
	X, y = array(X), array(y)
	return X, y

In [35]:
def invert(seq, alphabet):
	int_to_char = dict((i, c) for i, c in enumerate(alphabet))
	strings = list()
	for pattern in seq:
		string = int_to_char[argmax(pattern)]
		strings.append(string)
	return ''.join(strings)

In [36]:
result = model.predict(X, batch_size=n_batch, verbose=0)
# calculate error
expected = [invert(x, alphabet) for x in y]
predicted = [invert(x, alphabet) for x in result]
# show some examples
for i in range(1):
	print('Expected=%s, Predicted=%s' % (expected[i], predicted[i]))

Expected=12, Predicted=12
